Notes:


1. This is my attempt to reproduce some results from "A stochastic immersed boundary method for fluid-structure dynamics at microscopic length scales" (2007), by Atzberger, Kramer, & Peskin.


2. To calculate the diffusion constant to reproduce Fig.3, we need use
unwrapped coordinate for the single particle's position. We need to use the wrapped position to calculate the forces from the hydrodynamic field though.


3. The FFT and IFFT automatically satisfy PBC.


4. In this method, a single, isolated particle exerts no forces on the fluid field


5. Fluid grid cell edge lengths should be chosen smaller than the radii of the IB particles. 

6. Big picture view of sequence of events for a single time step:
    
    i. FFT the force density 
    
    ii. update the velocity field and $\Gamma_{\bf{k}}$ field at each point with a triple loop over all points in $k$-space
    
    iii. IFFT the $\Gamma_{\bf{k}}$ field to get the new IB positions in real space.

7. Equations $9-14$, $16-20$ can all be done locally, that is, inside the triple loop over all points in $k$-space

8. This means I need to rewrite the main function.

SIBM Algorithm steps for getting $n+1$-th fluid and IB node configuration from the $n$-th:

1. Have ${\bf{u}}_{\bf{k}}^n$ defined at each point in $k$-space, and ${\bf{X}}^n$ from the $n$-th time step, or from the initial conditions

2. Calculate ${\bf{f}}_{\bf{m}}^n$ from the IB node configuration ${\bf{X}}^n$, at each point in $m$-space

3. Calculate FFT ${\bf{f}}_{\bf{m}}^n$ to get ${\bf{f}}_{\bf{k}}^n$, at each point in $k$-space

4. Calculate ${\bf{\Xi}}_{\bf{k}}^n$ and ${\bf{G}}_{\bf{k}}^n$, at each point in $k$-space (note: this step could be done at any time between steps 1-3)

5. Calculate ${\bf{u}}_{\bf{k}}^{n+1}$ from ${\bf{\Xi}}_{\bf{k}}^n$ and ${\bf{G}}_{\bf{k}}^n$, at each point in $k$-space

6. Calculate ${\bf{\Gamma}}_{\bf{k}}^n$ from ${\bf{u}}_{\bf{k}}^n$, ${\bf{f}}_{\bf{k}}^n$, ${\bf{\Xi}}_{\bf{k}}^n$ and ${\bf{G}}_{\bf{k}}^n$, at each point in $k$-space 

7. Calculate IFFT ${\bf{\Gamma}}_{\bf{k}}^n$ to get ${\bf{\Gamma}}_{\bf{m}}^n$, at each point in $m$-space

8. Calculate ${\bf{X}}^{n+1}$ from ${\bf{\Gamma}}_{\bf{m}}^n$ (note: steps 4-8 could be done at any time after steps 4-5)

Need to fix so that $X^{n,j}$ follow PBC

In [ ]:
using MEDYAN
using StaticArrays
using LinearAlgebra
using Random
using SmallZarrGroups
using CairoMakie
using GLMakie
using IterativeSolvers
using FFTW
using DSP
using Plots
using OffsetArrays
using Statistics
#Random.seed!(1234);

In [ ]:
"""
Calculate the velocity field uk at the n+1-th timestep at all points in k-space. Also calculate the 
random variable Γm at the n-th timestep at all points in m-space.
"""
function update_fields(ukold,fmdensold,N,T,ρ,μ,Δx,Δt)
    #define arrays
    fkdensold = zeros(Complex{Float64},N[1],N[2],N[3],3)
    Γkold = zeros(Complex{Float64},N[1],N[2],N[3],3)
    Γmold = zeros(N[1],N[2],N[3],3)
    uknew = zeros(Complex{Float64},N[1],N[2],N[3],3)

    #insure grid array functions are indexed from 0,...,N-1
    fkdensold = OffsetArray(fkdensold,-1,-1,-1,0)
    Γkold = OffsetArray(Γkold,-1,-1,-1,0)
    Γmold = OffsetArray(Γmold,-1,-1,-1,0)
    uknew = OffsetArray(uknew,-1,-1,-1,0)
    
    #calculate FFT of discrete force density field
    fkdensold[:,:,:,1] = fft(view(fmdensold,:,:,:,1))
    fkdensold[:,:,:,2] = fft(view(fmdensold,:,:,:,2))
    fkdensold[:,:,:,3] = fft(view(fmdensold,:,:,:,3))
    
    #calculate discrete velocity field in k-space, skipping k=[0,0,0], according to remark after Eqn. 59
    for ik in 0:N[1]-1
        for jk in 0:N[2]-1
            for kk in 0:N[3]-1
                #need to use same value of Ξk to calculate Γk and uk, at particular k-space point
                #evolve k=(0,0,0) mode deterministically
                if (ik == 0) & (jk == 0) & (kk == 0)
                    uknew[ik,jk,kk,:] = view(uknew,ik,jk,kk,:) # + external force?
                
                #evolve all other modes stochastically
                else
                    Ξkold = calculate_Ξk([ik,jk,kk],N,T,ρ,μ,Δx,Δt)
                    Γkold[ik,jk,kk,:] = calculate_Γk(view(ukold,ik,jk,kk,:),view(fkdensold,ik,jk,kk,:),Ξkold,[ik,jk,kk],N,T,ρ,μ,Δx,Δt)
                    uknew[ik,jk,kk,:] = update_uk([ik,jk,kk],view(ukold,ik,jk,kk,:),view(fkdensold,ik,jk,kk,:),Ξkold,N,ρ,μ,Δx,Δt)
                end
            end
        end
    end
    
    #symmetrize the k-space field matrices
    for ik in 0:N[1]-1
        for jk in 0:N[2]-1
            for kk in 0:N[3]-1
                #get conjugate indices
                ikp = (N[1] - ik)%N[1]
                jkp = (N[2] - jk)%N[2] 
                kkp = (N[3] - kk)%N[3]
                
                #if k is a special point then field is real
                if (ik == ikp) & (jk == jkp) & (kk == kkp)
                    Γkold[ik,jk,kk,:] = real(view(Γkold,ik,jk,kk,:))
                    uknew[ik,jk,kk,:] = real(view(uknew,ik,jk,kk,:))
                    Γkold[ikp,jkp,kkp,:] = view(Γkold,ik,jk,kk,:)
                    uknew[ikp,jkp,kkp,:] = view(uknew,ik,jk,kk,:)
                
                #otherwise impose conjugate symmetry condition
                else
                    Γkold[ikp,jkp,kkp,:] = conj(view(Γkold,ik,jk,kk,:))
                    uknew[ikp,jkp,kkp,:] = conj(view(uknew,ik,jk,kk,:))
                end
            end
        end
    end
    
    #get r-space integrated velocity field
    Γmold[:,:,:,1] = prod(N)*ifft(OffsetArray(view(Γkold,:,:,:,1),+1,+1,+1))
    Γmold[:,:,:,2] = prod(N)*ifft(OffsetArray(view(Γkold,:,:,:,2),+1,+1,+1))
    Γmold[:,:,:,3] = prod(N)*ifft(OffsetArray(view(Γkold,:,:,:,3),+1,+1,+1))
    
    return uknew,Γmold
end

##########################################################################################################
#functions for initializing the harmonic polymer, in a uniformly random velocity field

"""
Initialize components of real space fluid velocity field as a uniformly random field.
"""
function initialize_um_field_randunif(N,randmag)
    um = (rand(N[1],N[2],N[3],3).-0.5)*randmag
    return um
end

"""
Initialize configuration of IB nodes for a single filament unbent, oriented
parallel to the z-axis.
"""
function initialize_IB_nodes_filparaz(nnode,Δz,Δx,N)
    rnodes = zeros(nnode,3)
    for k in 1:nnode
        rnodes[k,1] = (N[1]/2)*Δx + rand()
        rnodes[k,2] = (N[2]/2)*Δx + rand()
        rnodes[k,3] = (k-1)*Δz + (N[3]/2)*Δx - (nnode/4)*Δz + rand()
    end
    return rnodes
end

##########################################################################################################
#functions for harmonic polymer 

"""
Define harmonic force law.
"""
function tension_law(r,K0,S0)
    return K0*(r - S0)
end

"""
Calculate the dispacement vector between two position vectors.
"""
function calculate_forcevec(rveci,rvecj,K0,S0)
    dispvec = rvecj .- rveci
    return (dispvec./norm(dispvec))*tension_law(norm(dispvec),K0,S0)
end

"""
Calculate the array of forces on each node in the IB give the configuration of the nodes in the IB.
"""
function calculate_Fnodes(rnodes,K0,S0)
    Fnodes = zeros(size(rnodes))
    #sum over all pairs of particles in the polymer
    for ib in 1:size(rnodes,1)-1
        temp_arr = calculate_forcevec(rnodes[ib,:],rnodes[ib+1,:],K0,S0)
        Fnodes[ib,:] += temp_arr
        Fnodes[ib+1,:] -= temp_arr
    end
    return Fnodes
end
##########################################################################################################

"""
Eqn. 8, part a: Calculate the force density array associated with each IB node.
"""
function calculate_fdenstot(rnodes,Fnodes,N,Δx,a)
    fmdenstot = zeros(N[1],N[2],N[3],3)
    for i in 1:N[1]
        for j in 1:N[2]
            for k in 1:N[3]
                fmdenstot[i,j,k,:] = calculate_fdensijk([i,j,k]*Δx,rnodes,Fnodes,a)
            end
        end
    end
    return fmdenstot
end

"""
Eqn. 8, part b: Calculates the discrete force density field on the fluid grid
from all of the immersed boundary nodes at a single point rvec in real space.
"""
function calculate_fdensijk(rgridpnt,rnodes,Fnodes,a)
    fdensijk = zeros(3)
    #loop over all nodes in the immersed boundary
    for l in 1:size(rnodes,1)
        fdensijk += Fnodes[l,:]*δₐ(rgridpnt .- view(rnodes,l,:),a)
    end
    return fdensijk
end

"""
Eqn. 9: Updates FT of velocity field vector at a single point in k-space.
Returns three component vector at a single point in k-space.
"""
function update_uk(kvec,uk,fk,Ξk,N,ρ,μ,Δx,Δt)
    term_αk = αk(kvec,N,ρ,μ,Δx)
    term_Projk = Projorthogk(kvec,N,Δx)
    term1 = exp(-term_αk*Δt)*uk
    term2 = inv(ρ*term_αk)*(1 - exp(-term_αk*Δt))*(term_Projk*fk)
    term3 = term_Projk*Ξk
    return term1 .+ term2 .+ term3
end

"""
Eqn. 10: Fourier transform of nabla vector operator.
"""
function gk(kvec,N,Δx)
    return [sin(2π*kvec[1]/N[1]),sin(2π*kvec[2]/N[2]),sin(2π*kvec[3]/N[3])]./Δx
end

"""
Eqn. 11, part a: Generate Ξk random variable.
"""
function calculate_Ξk(kvec,N,T,ρ,μ,Δx,Δt)
    return sqrt(σ²k(kvec,N,T,ρ,μ,Δx,Δt))*[randn()+(im*randn()),randn()+(im*randn()),randn()+(im*randn())]
end

"""
Eqn. 11, part b: Variance of the random variable Ξk
"""
function σ²k(kvec,N,T,ρ,μ,Δx,Δt)
    term_αk = αk(kvec,N,ρ,μ,Δx)
    return Dk(kvec,N,T,ρ,μ,Δx)*inv(term_αk)*(1 - exp(-2*term_αk*Δt))
end
    
"""
Eqn. 12: Fourier transform of laplacian scalar operator.
"""
function αk(kvec,N,ρ,μ,Δx)
    return (2*μ*inv(ρ*Δx^2))*((1-cos(2π*kvec[1]/N[1]))+(1-cos(2π*kvec[2]/N[2]))+(1-cos(2π*kvec[3]/N[3])))
end

"""
Eqn. 13: Fourier transform of diffusion constant.
"""
function Dk(kvec,N,T,ρ,μ,Δx)
    kB = 8.316*1E3 #nm^2*amu/ns^2*K
    if (kvec[1] ∈ [0,N[1]/2]) && (kvec[2] ∈ [0,N[2]/2]) && (kvec[3] ∈ [0,N[3]/2])
        return (kB*T*inv(ρ*prod(N)*(Δx^3)))*αk(kvec,N,ρ,μ,Δx)
    else
        return (kB*T*inv(2*ρ*prod(N)*(Δx^3)))*αk(kvec,N,ρ,μ,Δx)
    end
end
    
"""
Eqn. 15: Update all IB node positions.
"""
function update_rnodes(rnodes,Γm,Δx,a)
    Δrnodes = zeros(size(rnodes,1),3)
    #integrate over all grid points
    for i in 0:N[1]-1
        for j in 0:N[2]-1
            for k in 0:N[3]-1
                for l in 1:size(rnodes,1)
                    δijk = δₐ([i,j,k]*Δx .- view(rnodes,l,:),a)
                    Δrnodes[l,:] += δijk*Γm[i,j,k,:]
                end
            end
        end
    end
    return rnodes .+ Δrnodes*(Δx^3)
end
    
"""
Eqn. 17: Γk random variable.
"""
function calculate_Γk(uk,fk,Ξk,kvec,N,T,ρ,μ,Δx,Δt)
    term1 = Hk(uk,fk,kvec,N,T,ρ,μ,Δx,Δt)
    term2 = c1k(kvec,N,ρ,μ,Δx,Δt)*(Projorthogk(kvec,N,Δx)*Ξk)
    term3 = c2k(kvec,N,ρ,μ,Δx,Δt)*(Projorthogk(kvec,N,Δx)*calculate_Gk())
    return term1 .+ term2 .+ term3 #, term2 .+ term3
end

"""
Eqn. 18: Hk random variable.
"""
function Hk(uk,fk,kvec,N,T,ρ,μ,Δx,Δt)
    term_αk = αk(kvec,N,ρ,μ,Δx)
    term1 = inv(term_αk)*(1-exp(-term_αk*Δt))*uk
    term2 = ((Δt*inv(term_αk))+(inv(term_αk^2)*(exp(-term_αk*Δt)-1)))*inv(ρ)*(Projorthogk(kvec,N,Δx)*fk)
    return term1 .+ term2
end

"""
Generate Gk complex random variable.
"""
function calculate_Gk()
    return [randn()+(im*randn()),randn()+(im*randn()),randn()+(im*randn())]
end

"""
Eqn. 19: Generate first correlation coefficient.
"""
function c1k(kvec,N,ρ,μ,Δx,Δt)
    term_αk = αk(kvec,N,ρ,μ,Δx)
    return inv(term_αk)*tanh(0.5*term_αk*Δt)
end

"""
Eqn. 20: Generate second correlation coefficient.
"""
function c2k(kvec,N,ρ,μ,Δx,Δt)
    term_αk = αk(kvec,N,ρ,μ,Δx)
    return sqrt(inv(term_αk^3)*(2*Dk(kvec,N,T,ρ,μ,Δx))*(term_αk*Δt - 2*tanh(0.5*term_αk*Δt)))
end

"""
Eqn. 33: Projection operator projects out the part of the field orthogonal to gk field operator.
"""
function Projorthogk(kvec,N,Δx)
    return Matrix(1.0I, 3, 3) - gk(kvec,N,Δx).*transpose(gk(kvec,N,Δx))*inv(sum(gk(kvec,N,Δx).^2))
end

"""
Eqn. 104: Numerical estimator of diffusion coefficient for single particle measured from simulation
of a trajectory. Results are then averaged over many trajectories to get Eqn. 104.
"""
function D_estimate(nstep,Δt,X)
    return inv(6*nstep*Δt)*(norm(view(X,nstep,:).-view(X,1,:)))^2
end
    
"""
Eqn. A.1: Define the 1D radially dependent Peskin kernel component.
"""
function ϕ(r)
    if r <= -2
        return 0
    elseif -2 <= r < -1
        return inv(8)*(5 + 2*r - sqrt(-7 - 12*r - 4*r^2))
    elseif -1 <= r < 0
        return inv(8)*(3 + 2*r + sqrt(1 - 4*r - 4*r^2)) 
    elseif 0 <= r < 1
        return inv(8)*(3 - 2*r + sqrt(1 + 4*r - 4*r^2)) 
    elseif 1 <= r < 2
        return inv(8)*(5 - 2*r - sqrt(-7 + 12*r - 4*r^2)) 
    elseif 2 <= r
        return 0
    end
end

"""
Eqn. A.2: Define the 3D radially dependent Peskin kernel.
"""
function δₐ(rvec,a)
    return inv(a^3)*ϕ(inv(a)*rvec[1])*ϕ(inv(a)*rvec[2])*ϕ(inv(a)*rvec[3])
end

"""
Move entire system nstep timesteps forward
"""
function main_single_diffusion(N,T,ρ,μ,Δx,Δt,nstep,a,randmag)
    #initialize velocity field and IB node positions
    uk = zeros(Complex{Float64},N[1],N[2],N[3],3)
    um = initialize_um_field_randunif(N,randmag)
    
    #insure grid array functions are indexed from 0,...,N-1
    uk = OffsetArray(uk,-1,-1,-1,0)
    um = OffsetArray(um,-1,-1,-1,0)
    
    #FFT initial velocity field
    uk[:,:,:,1] = fft(view(um,:,:,:,1))
    uk[:,:,:,2] = fft(view(um,:,:,:,2))
    uk[:,:,:,3] = fft(view(um,:,:,:,3))
    
    #initialize the single diffusing particle's IB node location
    #to be the center of the simulation domain
    rnodes = [N[1]*Δx/2 N[2]*Δx/2 N[3]*Δx/2]
    
    #initialize the force on the nodes, which in this special 
    #case is zero since there is only a single point
    Fnodes = [0.0 0.0 0.0]
    
    #initialize output IB trajectory data arrays
    traj_rnodes = rnodes
    
    #integrate system forward in time
    for it in 1:nstep
        fmdens = calculate_fdenstot(rnodes,Fnodes,N,Δx,a)
        uk,Γm = update_fields(uk,fmdens,N,T,ρ,μ,Δx,Δt)
        rnodes = update_rnodes(rnodes,Γm,Δx,a)
        traj_rnodes = cat(traj_rnodes,rnodes,dims=1)
    end
    return traj_rnodes
end

"""
Estimate statistics for diffusion coefficient for range of IB point radii
"""
function main_estimate_D(N,T,ρ,μ,Δx,Δt,nstep,a,randmag,nrep)
    D_est = []
    D_est_stderr = []
    #D_std = []
    for ia in 1:length(a)
        D_sample = []
        for ir in 1:nrep
            traj_rnodes = main_single_diffusion(N,T,ρ,μ,Δx,Δt,nstep,a[ia],randmag)
            D_sample = cat(D_sample,D_estimate(nstep,Δt,traj_rnodes),dims=1)
        end
        D_est = cat(D_est,mean(D_sample),dims=1)
        D_est_stderr = cat(D_est_stderr,sqrt(var(D_sample)/nrep),dims=1)
        #D_std = cat(D_std,std(D_sample),dims=1)
    end
    return D_est,D_est_stderr
end

"""
Exact expression for D
"""
function calc_D_exact(N,T,ρ,μ,Δx,a)
    kB = 8.316*1E3 #nm^2*amu/ns^2*K
    
    δ0 = OffsetArray(zeros(N[1],N[2],N[3]),-1,-1,-1)
    #integrate over all grid points in r-space
    for i in 0:N[1]-1
        for j in 0:N[2]-1
            for k in 0:N[3]-1
                #use (x,y,z) = (0,0,0) point
                δ0[i,j,k] = δₐ([i,j,k]*Δx,a)
            end
        end
    end
    #get Fourier transform of kernel
    δₐₖ = OffsetArray(fft(δ0),-1,-1,-1)
    
    D = 0.0
    #integrate over all grid points in k-space
    for ik in 0:N[1]-1
        for jk in 0:N[2]-1
            for kk in 0:N[3]-1
                if (ik ∈ [0,N[1]/2]) && (jk ∈ [0,N[2]/2]) && (kk ∈ [0,N[3]/2])
                    if (ik != 0) || (jk != 0) || (kk != 0)
                        D += 3*abs2(δₐₖ[ik,jk,kk])*inv(αk([ik,jk,kk],N,ρ,μ,Δx))
                    end
                else
                    D += 2*abs2(δₐₖ[ik,jk,kk])*inv(αk([ik,jk,kk],N,ρ,μ,Δx))
                end
            end
        end
    end
    D *= kB*T*prod(N)*(Δx^3)*inv(3*ρ)
    
    return D
end

"""
Move entire system nstep timesteps forward
"""
function main_single_diffusion(N,T,ρ,μ,Δx,Δt,nstep,a,randmag)
    #initialize velocity field and IB node positions
    uk = zeros(Complex{Float64},N[1],N[2],N[3],3)
    um = initialize_um_field_randunif(N,randmag)
    
    #insure grid array functions are indexed from 0,...,N-1
    uk = OffsetArray(uk,-1,-1,-1,0)
    um = OffsetArray(um,-1,-1,-1,0)
    
    #FFT initial velocity field
    uk[:,:,:,1] = fft(view(um,:,:,:,1))
    uk[:,:,:,2] = fft(view(um,:,:,:,2))
    uk[:,:,:,3] = fft(view(um,:,:,:,3))
    
    #initalize velocity data array
    total_um = uk
    
    #initialize the single diffusing particle's IB node location
    #to be the center of the simulation domain
    rnodes = [N[1]*Δx/2 N[2]*Δx/2 N[3]*Δx/2]
    
    #initialize the force on the nodes, which in this special 
    #case is zero since there is only a single point
    Fnodes = [0.0 0.0 0.0]
    
    #initialize output IB trajectory data arrays
    traj_rnodes = rnodes
    
    #integrate system forward in time
    for it in 1:nstep
        fmdens = calculate_fdenstot(rnodes,Fnodes,N,Δx,a)
        uk,Γm = update_fields(uk,fmdens,N,T,ρ,μ,Δx,Δt)
        
        #get r-space velocity field
        um[:,:,:,1] = prod(N)*ifft(OffsetArray(view(uk,:,:,:,1),+1,+1,+1))
        um[:,:,:,2] = prod(N)*ifft(OffsetArray(view(uk,:,:,:,2),+1,+1,+1))
        um[:,:,:,3] = prod(N)*ifft(OffsetArray(view(uk,:,:,:,3),+1,+1,+1))
        total_um = cat(total_um,um,dims=5)
        
        rnodes = update_rnodes(rnodes,Γm,Δx,a)
        traj_rnodes = cat(traj_rnodes,rnodes,dims=1)
    end
    return traj_rnodes,total_um
end

"""
Animate the fluid velocity field and the trajectory of the immersed boundary particle
"""
function animate_ufield_IBtraj(framerate,grid_skip)
    
    filename = "ufield_IBtraj_anim.mp4"
    
    ps = [Point3f(x, y, z) for x in 1:grid_skip:N[1] for y in 1:grid_skip:N[2] for z in 1:grid_skip:N[3]]

    time = Observable(1)

    ufield = @lift(ns = map(
        p -> Vec3f(total_um[Int(p[1]),Int(p[2]),Int(p[3]),1,$time],
        total_um[Int(p[1]),Int(p[2]),Int(p[3]),2,$time],
        total_um[Int(p[1]),Int(p[2]),Int(p[3]),3,$time]),
        ps))

    IBtrajx = @lift(traj_rnodes[$time,1])
    IBtrajy = @lift(traj_rnodes[$time,2])
    IBtrajz = @lift(traj_rnodes[$time,3])
    
    fig = 
    GLMakie.arrows(
        ps*Δx, ufield, fxaa=true, # turn on anti-aliasing
        linecolor = :gray, arrowcolor = :black,
        linewidth = 5.0, lengthscale = 100.0, arrowsize = Vec3f(10.0, 10.0, 10.0),
        axis=(type=Axis3, title = @lift("t = $(round($time*Δt, digits = 5)) ns"))
    )

    GLMakie.scatter!(IBtrajx,IBtrajy,IBtrajz,markersize=a[ia],color=:blue,alpha=0.8)
    GLMakie.scatter!(IBtrajx,IBtrajy,IBtrajz,markersize=5,color=:red,alpha=1.0)

    GLMakie.xlims!(0.0,1000.0)
    GLMakie.ylims!(0.0,1000.0)
    GLMakie.zlims!(0.0,1000.0)


    fig.axis.xlabel = L"$x \quad [\text{nm}]$"
    fig.axis.ylabel = L"$y \quad [\text{nm}]$"
    fig.axis.zlabel = L"$z \quad [\text{nm}]$"

    #axislegend(position = :rb)
    timestamps = range(1, nstep, step=1)
    record(fig, filename, timestamps;framerate = framerate) do t
        time[] = t
    end
end

"""
Plot estimate of the diffusion coefficient with standard errors as a function of 
the immersed boundary particle radius parameter a
"""
function plot_D_est(D_conversion_factor)
    f = Figure(fontsize = 20)
    ax = Axis(f[1, 1])
    
    #plot numerical results
    CairoMakie.errorbars!(a/Δx, D_est*D_conversion_factor,
        D_est_stderr*D_conversion_factor, color = :red, whiskerwidth = 10)
    
    CairoMakie.scatter!(a/Δx, D_est*D_conversion_factor, markersize = 10, color = :black)
    
    #plot analytical results
    #CairoMakie.scatter!(a/Δx, D_exact, markersize = 15, color = :blue)
    #ax.title = ""
    ax.xlabel = L"$a \quad [\text{meshwidths}]$"
    ax.ylabel = L"$D \quad [{\mu m}^{2}/{s}]$"
    f
    save("D_est_vs_a_samp_size_$nrep.pdf", f; backend=CairoMakie)
    f
end

In [ ]:
nrep = 100
N = [32,32,32]
L = 1.0*10^3 #nm
T = 300.0 #K
ρ = 602.0 #amu/nm^3
μ = 6.02*10^5 #amu/(nm ns)
Δx = L/N[1] #nm
Δt = 1.0*10^3 #ns
nstep = 10
k₀ = [1,2,3,4,5]
a = k₀*Δx #nm
randmag = 0.0
ia = 1
D_conversion_factor = 1000
framerate = 10
grid_skip = 4

#D_exact = calc_D_exact(N,T,ρ,μ,Δx,a[ia])
#D_est,D_est_stderr = main_estimate_D(N,T,ρ,μ,Δx,Δt,nstep,a,randmag,nrep)
traj_rnodes,total_um = main_single_diffusion(N,T,ρ,μ,Δx,Δt,nstep,a[ia],randmag)
animate_ufield_IBtraj(framerate,grid_skip)